# This notebook is used to create the data clothes csv file given a folder of images

In [21]:
%load_ext autoreload
%autoreload 2

In [22]:
import sys
import matplotlib.pyplot as plt
import scripts.main_utilities as mu
import roboflow
import pandas as pd
import numpy as np
from roboflow import Roboflow
import os
import torch
from ultralytics import YOLO
import configparser

# Read config file
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

# Var

In [23]:
PATH_IMAGES_FOLDER  = config["PATH"]["PATH_IMAGES_FOLDER"]
PATH_DATA_COLOR     = config["PATH"]["PATH_DATA_COLOR"]
PATH_SAVE_DATA      = config["PATH"]["PATH_SAVE_DATA"]

YOLO_MODEL          = config["MODEL"]["YOLO_MODEL"]
ROBOFLOW_MODEL      = config["MODEL"]["ROBOFLOW_MODEL"]

EXTENSION_IMG       = ["jpeg", 'png']

# Model

In [13]:
device = torch.device('cpu')
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
MODEL_DETECT_OBJ    = YOLO(YOLO_MODEL)
MODEL_DETECT_OBJ.to(device) 

# Connect to Roboflow
roboflow.login()
rf = Roboflow()

# Load pretrained model for clothes recognition
project = rf.workspace().project(ROBOFLOW_MODEL)
fashion = project.version(4).model

You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...


# Extract images of the folder

In [14]:
path_img_list, img_names = mu.get_extension_folder(PATH_IMAGES_FOLDER, EXTENSION_IMG)
path_img_list

['static/images/asap_rocky_outfit.jpeg',
 'static/images/asap_rocky_outfit7.jpeg',
 'static/images/outfit1.jpeg',
 'static/images/asap_rocky_outfit10.jpeg',
 'static/images/asap_rocky_outfit11.jpeg',
 'static/images/asap1.jpeg',
 'static/images/asap_rocky_outfit1.jpeg',
 'static/images/eminem.jpeg']

In [24]:
PATH_DATA_COLOR

'static/data_color.csv'

# Load color dataframe

In [25]:
data_color = pd.read_csv(PATH_DATA_COLOR)
data_color.head()

,color,R,G,B
0,Yellow,255,255,0
1,Red,255,0,0
2,Blue,0,0,255
3,Green,0,255,0
4,Orange,255,165,0


# Load img path with plt.imread()

In [26]:
img_list = [plt.imread(i) for i in path_img_list]

# Extract data clothes from img

In [27]:
data_clothes_ = mu.detect_outfit_color_batch(img_list, img_names, MODEL_DETECT_OBJ, fashion, data_color)
data_clothes_.head()

[INFO] : 1 person found
[INFO] : 0 outfit found
[INFO] : 4 person found
[INFO] : 0 outfit found
[INFO] : 0 outfit found
[INFO] : 0 outfit found
[INFO] : 1 outfit found
[INFO] : 2 person found


X does not have valid feature names, but NearestNeighbors was fitted with feature names


[INFO] : 1 outfit found


X does not have valid feature names, but NearestNeighbors was fitted with feature names


[INFO] : 0 outfit found
[INFO] : 2 person found
[INFO] : 2 outfit found


...

X does not have valid feature names, but NearestNeighbors was fitted with feature names


[INFO] : 0 outfit found
[INFO] : 2 person found
[INFO] : 1 outfit found


X does not have valid feature names, but NearestNeighbors was fitted with feature names


[INFO] : 0 outfit found
[INFO] : 1 person found
[INFO] : 0 outfit found
[INFO] : 3 person found
[INFO] : 0 outfit found
[INFO] : 0 outfit found
[INFO] : 0 outfit found


,id,bottom,bottom_color,top,top_color
0,asap_rocky_outfit7.jpeg,skirt,silver,NaN,NaN
0,outfit1.jpeg,pants,grey,NaN,NaN
0,asap_rocky_outfit10.jpeg,skirt,black,polar,black
0,asap_rocky_outfit10.jpeg,NaN,NaN,polar,black
0,asap1.jpeg,NaN,NaN,polar,black


# Save the data csv

In [28]:
data_clothes_.to_csv(PATH_SAVE_DATA, index= False)